In [ ]:
import numpy as np
import pandas as pd
from time import time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

%matplotlib inline

In [ ]:
all_events=pd.read_csv('raw_data/tianchi_fresh_comp_train_user.csv')
all_items=pd.read_csv('raw_data/tianchi_fresh_comp_train_item.csv')
target_items=set(all_items.item_id)
action_types=['browsed','collected','carted','bought']

In [ ]:
all_users=all_events.user_id.unique()
sample_users=all_users[:500]
sample_events=all_events[all_events.user_id.apply(lambda uid:uid in sample_users)]
sample_events['time']=pd.DatetimeIndex(sample_events.time)

behaves=sample_events[['user_id','item_id','behavior_type','time']].pivot_table(index=['user_id','item_id'],columns='behavior_type',aggfunc='count').fillna(0)
behaves.columns=action_types
behaves['label']=behaves.bought.apply(lambda b:1. if b>0 else 0.)

buy_behaves=behaves[behaves.label==1]
nbuy_behaves=behaves[behaves.label==0]


In [ ]:
sample_nbuy_idx=np.random.choice(len(nbuy_behaves),len(buy_behaves))

sample_behaves=pd.concat([buy_behaves,nbuy_behaves.iloc[sample_nbuy_idx]])

assert(len(sample_behaves[sample_behaves.label==0])==len(sample_behaves[sample_behaves.label==1]))

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(sample_behaves[['browsed']],sample_behaves[['label']],test_size=.2)

lr=LogisticRegression()
lr.fit(X_train,y_train)
roc_auc_score(y_train,lr.predict_proba(X_train)[:,1])